In [1]:
import torch.nn.functional as F
import torch
import numpy as np
from pathlib import Path
import plotly.graph_objs as go


from initialization import NeuralSurfaceReconstruction

In [2]:
nsr = NeuralSurfaceReconstruction(
    data=Path("../data/calculator"),
)

In [3]:
nsr.fg_V.feat

tensor([[[[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           ...,
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],

          [[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           ...,
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],

          [[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
           [0.5000, 0.5000

In [32]:
tp, st, v = nsr.query_points_and_direction(0, N_query_points=100, t_size=0.15,
                                        _pixel_grid_width_step=100.0,
                                        _pixel_grid_height_step=100.0,
                                        _scale=1.0)

In [33]:
st_i = st[:, 50]

In [34]:
is_f, is_b = nsr.model.sector(st_i)

norm_fx_i = ((st_i[is_f]-nsr.model.fg_V.center)/nsr.model.fg_V.scale).float()
norm_fx_i = torch.flip(  # X Y Z -> Z Y X
    norm_fx_i[torch.newaxis, torch.newaxis, torch.newaxis],
    dims=[-1]
)

In [35]:
norm_fx_i

tensor([[[[[-0.7026, -0.9388, -0.9761],
           [-0.7037, -0.8551, -0.9738],
           [-0.7048, -0.7723, -0.9717],
           ...,
           [-0.7432,  0.8019,  0.3036],
           [-0.7442,  0.8825,  0.3060],
           [-0.7453,  0.9636,  0.3086]]]]])

In [36]:
f_feat_i = F.grid_sample(
    nsr.model.fg_V.feat, norm_fx_i,
    mode="bilinear", padding_mode="border", align_corners=True
).squeeze()  # CxN

In [37]:
nsr.model.fg_V.feat.shape, norm_fx_i.shape, f_feat_i.shape

(torch.Size([1, 3, 16, 16, 16]),
 torch.Size([1, 1, 1, 425, 3]),
 torch.Size([3, 425]))

In [38]:
torch.cat([f_feat_i, v[:, is_f]], axis=0)

tensor([[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [1.7510, 1.7370, 1.7231,  ..., 1.4537, 1.4400, 1.4263],
        [1.6374, 1.6370, 1.6367,  ..., 1.4165, 1.4161, 1.4156]])

In [39]:
v

tensor([[1.8223, 1.8079, 1.7936,  ..., 1.3285, 1.3143, 1.2999],
        [1.7108, 1.7100, 1.7092,  ..., 1.4112, 1.4105, 1.4096]])

In [41]:
L_o = nsr.model.L_o(st_i, v)

In [43]:
L_o.shape

torch.Size([858, 3])

In [47]:
st_i.shape[0]

858

In [48]:
torch.full((st_i.shape[0],), 0.0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [22]:
go.Figure(data=[
    go.Scatter3d(
        x=nsr.fg_V.grid_pos[0].flatten(),
        y=nsr.fg_V.grid_pos[1].flatten(),
        z=nsr.fg_V.grid_pos[2].flatten(),
        mode='markers',
        marker={
            'size': 10,
            "color": nsr.fg_V.sdf.flatten(),
            'opacity': 0.3,
            "cmax": 1.0,
            "cmin": -1.0,
            "colorbar": {
                "thickness": 10,
            }
        }
    ),
    go.Scatter3d(
        x=nsr.bg_V.grid_pos[0].flatten(),
        y=nsr.bg_V.grid_pos[1].flatten(),
        z=nsr.bg_V.grid_pos[2].flatten(),
        mode='markers',
        marker={
            'size': 10,
            "color": nsr.bg_V.sdf.flatten(),
            'opacity': 0.2,
            "cmax": 1.0,
            "cmin": -1.0,
            # "colorbar": {
            #     "thickness": 10,
            # }
        }
    ),
    go.Scatter3d(
        x=st_i[:, 0],
        y=st_i[:, 1],
        z=st_i[:, 2],
        mode='markers',
        marker={
            'size': 3,
            "color": sdf_i,
            'opacity': 1.0,
            "cmax": 1.0,
            "cmin": -1.0,
        }
    ),
], layout=go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(
        aspectmode="cube",
        xaxis=dict(range=[-30, 30]),  # Set x-axis range
        yaxis=dict(range=[-30, 30]),  # Set y-axis range
        zaxis=dict(range=[-30, 30])   # Set z-axis range
    ),
    scene_camera= dict(
        eye=dict(x=0.0, y=0.0, z=0.1)
    )
))

In [4]:
_, st, v = nsr.query_points_and_direction(700, N_query_points=100, t_size=0.15,
                                        _pixel_grid_width_step=100.0,
                                        _pixel_grid_height_step=100.0,
                                        _scale=1.0)
stx, sty, stz = np.hsplit(st.reshape(-1, 3), 3)

In [5]:
st_i = st[:, -1]

In [6]:
x_i = torch.tensor(st_i).float()
sdf_i = nsr.model.S(x_i)

In [7]:
go.Figure(data=[
    go.Scatter3d(
        x=nsr.fg_V.grid_pos[0].flatten(),
        y=nsr.fg_V.grid_pos[1].flatten(),
        z=nsr.fg_V.grid_pos[2].flatten(),
        mode='markers',
        marker={
            'size': 10,
            "color": nsr.fg_V.sdf.flatten(),
            'opacity': 0.3,
            "cmax": 1.0,
            "cmin": -1.0,
            "colorbar": {
                "thickness": 10,
            }
        }
    ),
    go.Scatter3d(
        x=nsr.bg_V.grid_pos[0].flatten(),
        y=nsr.bg_V.grid_pos[1].flatten(),
        z=nsr.bg_V.grid_pos[2].flatten(),
        mode='markers',
        marker={
            'size': 10,
            "color": nsr.bg_V.sdf.flatten(),
            'opacity': 0.2,
            "cmax": 1.0,
            "cmin": -1.0,
            "colorbar": {
                "thickness": 10,
            }
        }
    ),
    go.Scatter3d(
        x=st_i[:, 0],
        y=st_i[:, 1],
        z=st_i[:, 2],
        mode='markers',
        marker={
            'size': 3,
            "color": sdf_i,
            'opacity': 1.0,
            "cmax": 1.0,
            "cmin": -1.0,
        }
    ),
], layout=go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(
        aspectmode="cube",
        xaxis=dict(range=[-30, 30]),  # Set x-axis range
        yaxis=dict(range=[-30, 30]),  # Set y-axis range
        zaxis=dict(range=[-30, 30])   # Set z-axis range
    ),
    scene_camera= dict(
        eye=dict(x=0.0, y=0.0, z=0.1)
    )
))

In [13]:
_, st, v = nsr.query_points_and_direction(450, N_query_points=100, t_size=0.15,
                                        _pixel_grid_width_step=100.0,
                                        _pixel_grid_height_step=100.0,
                                        _scale=1.0)
# stx, sty, stz = np.hsplit(sto.reshape(-1, 3), 3)

In [14]:
# sto_0 = sto[:, 0]
# sto_0_norm = sto_0 / np.linalg.norm(sto_0, axis=1)[:, np.newaxis]
st_i = st[:, 0]

In [19]:
go.Figure(data=[
    go.Scatter3d(
        x=[0.0],
        y=[0.0],
        z=[0.0],
        mode='markers',
        marker={
            'size': 5,
            'opacity': 1.0,
        }
    ),
    go.Scatter3d(
        x=st_i[:, 0],
        y=st_i[:, 1],
        z=st_i[:, 2],
        mode='markers',
        marker={
            'size': 3,
            'opacity': 1.0,
            "color": v[:, 1],
            "colorbar": {
                "thickness": 10,
            }
        }
    ),
    # go.Scatter3d(
    #     x=sto_0_norm[:, 0],
    #     y=sto_0_norm[:, 1],
    #     z=sto_0_norm[:, 2],
    #     mode='markers',
    #     marker={
    #         'size': 3,
    #         'opacity': 1.0,
    #         "color": phi,
    #         "colorbar": {
    #             "thickness": 10,
    #         }
    #     }
    # ),
], layout=go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(
        aspectmode="cube",
        xaxis=dict(range=[-20, 20]),  # Set x-axis range
        yaxis=dict(range=[-20, 20]),  # Set y-axis range
        zaxis=dict(range=[-20, 20]),  # Set z-axis range
    ),
    scene_camera= dict(
        eye=dict(x=0.0, y=0.0, z=1.0)
    )
))

In [61]:
sto_0_norm = sto_0 / np.linalg.norm(sto_0, axis=1)[:, np.newaxis]
sto_0_norm

array([[-0.13511935, -0.24882095,  0.95907815],
       [-0.13482491, -0.23487704,  0.96262922],
       [-0.13453331, -0.22096313,  0.96595863],
       ...,
       [ 0.1542447 ,  0.23989184,  0.95846777],
       [ 0.15443019,  0.25370649,  0.95487399],
       [ 0.15461277,  0.26754722,  0.95105908]])

In [80]:
# https://en.wikipedia.org/wiki/Spherical_coordinate_system
# theta: inclination (0-180deg), 0deg = +Y; 180deg = -Y
theta = np.arccos(sto_0[:,1] / np.linalg.norm(sto_0, axis=1))*(180.0/np.pi)

In [81]:
# phi: azimuth (0-360deg), 0deg = +X 0Z; 90deg = 0X +Z; 180deg = -X 0Z
phi = np.sign(sto_0[:,2])*np.arccos(sto_0[:,0] / np.linalg.norm(sto_0[:, [0,2]], axis=1))*(180.0/np.pi)

In [92]:
vt = np.stack([theta, phi], axis=1)

In [87]:
theta[:5], phi[:5]

(array([104.40775291, 103.58437492, 102.76560883, 101.95136695,
        101.14153442]),
 array([98.01931416, 97.97292733, 97.928832  , 97.88702861, 97.84751747]))

In [93]:
len(theta), len(phi), len(sto), sto.shape, len(vt)

(858, 858, 858, (858, 100, 3), 858)